In [1]:
import os
from warnings import filterwarnings
from dotenv import load_dotenv, find_dotenv
filterwarnings(action="ignore")

from operator import itemgetter
from collections import defaultdict
from IPython.display import Markdown

from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import trim_messages, HumanMessage
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain, create_history_aware_retriever
from langchain_core.runnables import RunnableWithMessageHistory, RunnablePassthrough

load_dotenv(find_dotenv())

True

In [2]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [3]:
groq_model = ChatGroq(model="llama-3.2-90b-vision-preview")
groq_model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000225EDD577D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000225EDD74F90>, model_name='llama-3.2-90b-vision-preview', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
webbased_pdf_path = "https://dpvipracollege.ac.in/wp-content/uploads/2022/12/h.k.dass_.pdf"

In [5]:
pdf_loader = PyPDFLoader(file_path=webbased_pdf_path)
loaded_pdf = pdf_loader.load()

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=128000, chunk_overlap=100)
splitted_pdf_doc = text_splitter.split_documents(loaded_pdf)

In [7]:
model_embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
model_embedding

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [23]:
vector_store = Chroma.from_documents(documents=splitted_pdf_doc, embedding=model_embedding)

In [24]:
vectorstore_retrieval = vector_store.as_retriever(search_type="mmr", 
                                                  search_kwargs=dict(k=5, fetch_k=1000))
vectorstore_retrieval

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x0000022585299C50>, search_type='mmr', search_kwargs={'k': 5, 'fetch_k': 1000})

In [25]:
model_prompt = ChatPromptTemplate.from_messages([
                        ("system", "You are a helpful advanced engineering maths teacher. "
                                   "learn from the received context and use the knowledge to answer the question showing full workings and stating the final answer"
                                   "if you don't know the answer, provide a consise response(not more than 3 sentence) stating that you don't know it "
                                   "while suggesting relevant alternative to help the student."
                                   "Ensure your solution is in markdown format and is based in the context.\n\n"
                                   "{context}"),
                        MessagesPlaceholder("chat_history"),
                        ("human", "{input}")
                    ])

model_prompt

ChatPromptTemplate(input_variables=['chat_history', 'context', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.

In [26]:
history_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Given a chat history and the latest user question "
                    "which might reference context in the chat history, "
                    "formulate a standalone question which can be understood "
                    "without the chat history. Do NOT answer the question, "
                    "just reformulate it if needed and otherwise return it as is."),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_prompt

ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[l

In [27]:
history_aware_bot = create_history_aware_retriever(llm=groq_model, retriever=vectorstore_retrieval, prompt=history_prompt)
qa_chain = create_stuff_documents_chain(llm=groq_model, prompt=model_prompt)
math_model = create_retrieval_chain(retriever=history_aware_bot, combine_docs_chain=qa_chain)

math_model

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x0000022585299C50>, search_type='mmr', search_kwargs={'k': 5, 'fetch_k': 1000}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.too

In [28]:
class BotMemory:
    def __init__(self):
        self.storage = defaultdict(ChatMessageHistory)

    def CheckClientId(self, session_id:str)->BaseChatMessageHistory:
        return self.storage[session_id]

In [29]:
bot_memory = BotMemory()

In [43]:
math_bot_with_history = RunnableWithMessageHistory(runnable=math_model, 
                                                   get_session_history=bot_memory.CheckClientId,
                                                   input_messages_key="input",
                                                   history_messages_key="chat_history",
                                                   output_messages_key="answer"
                                                   )

math_bot_with_history

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  chat_history: RunnableBinding(bound=RunnableLambda(_enter_history), kwargs={}, config={'run_name': 'load_history'}, config_factories=[])
}), kwargs={}, config={'run_name': 'insert_history'}, config_factories=[])
| RunnableBinding(bound=RunnableLambda(_call_runnable_sync), kwargs={}, config={'run_name': 'check_sync_or_async'}, config_factories=[]), kwargs={}, config={'run_name': 'RunnableWithMessageHistory'}, config_factories=[]), kwargs={}, config={}, config_factories=[], get_session_history=<bound method BotMemory.CheckClientId of <__main__.BotMemory object at 0x0000022588EA0490>>, input_messages_key='input', output_messages_key='answer', history_messages_key='chat_history', history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)])

In [45]:
while True:
    user_input = input("User:")
    if user_input != "end":                
        bot_response = math_bot_with_history.invoke(
                                                  {"input": user_input}, 
                                                  config={"configurable": {"session_id": "Eddy"}}
                                                  )
        print(f"Bot: {bot_response['answer']}", end="\n\n")
    else:
        break

User: Hello, my name is Edifon Emmanuel Jimmy, a 500level student of Electrical Electronics Engineering University ofBenin


Bot: Hello Edifon Emmanuel Jimmy, nice to meet you. I'm your advanced engineering mathematics teacher. I see you're a 500-level student of Electrical Electronics Engineering at the University of Benin. That's great. I'm here to help you with any questions or problems you have in your advanced engineering mathematics course. What topic are you currently studying or what specific problem are you struggling with?



User: whats my name?


Bot: Your name is Edifon Emmanuel Jimmy.



User: Whats my level


Bot: You're a 500-level student.



User: ok


Bot: You seem ready to dive into some advanced engineering mathematics. What topic or problem would you like to tackle first?



User: end


In [47]:
bot_response

{'input': 'ok',
 'chat_history': [HumanMessage(content='Hello, my name is Edifon Emmanuel Jimmy, a 500level student of Electrical Electronics Engineering University ofBenin', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hello Edifon Emmanuel Jimmy, nice to meet you. I'm your advanced engineering mathematics teacher. I see you're a 500-level student of Electrical Electronics Engineering at the University of Benin. That's great. I'm here to help you with any questions or problems you have in your advanced engineering mathematics course. What topic are you currently studying or what specific problem are you struggling with?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='whats my name?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is Edifon Emmanuel Jimmy.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Whats my level', additional_kwargs={}, response_metadata={}),
  AIMessage(content="You're